In [ ]:
import requests
import pandas as pd

# Fetch the JSON data
url = "https://cdn.nba.com/static/json/staticData/scheduleLeagueV2.json"
response = requests.get(url)
data = response.json()
print(data.keys())
# Extract games data
games_data = []
for game_date in data["leagueSchedule"]["gameDates"]:
    game_date_str = game_date["gameDate"]
    for game in game_date["games"]:
        game_info = {
            "gameId": game["gameId"],
            "gameCode": game["gameCode"],
            "gameStatus": game["gameStatus"],
            "gameStatusText": game["gameStatusText"],
            "gameDate": game_date_str,
            "arenaName": game["arenaName"],
            "arenaCity": game["arenaCity"],
            "homeTeam": game["homeTeam"]["teamName"],
            "homeTeamCity": game["homeTeam"]["teamCity"],
            "homeTeamScore": game["homeTeam"]["score"],
            "awayTeam": game["awayTeam"]["teamName"],
            "awayTeamCity": game["awayTeam"]["teamCity"],
            "awayTeamScore": game["awayTeam"]["score"],
            "pointsLeaderName": f"{game['pointsLeaders'][0]['firstName']} {game['pointsLeaders'][0]['lastName']}" if game["pointsLeaders"] else None,
            "pointsLeaderPoints": game["pointsLeaders"][0]["points"] if game["pointsLeaders"] else None
        }
        games_data.append(game_info)

# Convert to DataFrame
df = pd.DataFrame(games_data)

df=df[df.gameId.str[0:3]!='001']

# Display the DataFrame
print(df.columns)


df.to_csv('schedule.csv')

print(df[df.gameId=='0032400001'])
df['gameDate'] = pd.to_datetime(df['gameDate']).dt.strftime('%Y-%m-%d')

df=df[df['gameDate']>='10222024']

team_acronyms = {
    'Celtics': 'BOS',
    'Lakers': 'LAL',
    'Pistons': 'DET',
    'Hawks': 'ATL',
    'Heat': 'MIA',
    '76ers': 'PHI',
    'Raptors': 'TOR',
    'Rockets': 'HOU',
    'Pelicans': 'NOP',
    'Jazz': 'UTA',
    'Clippers': 'LAC',
    'Trail Blazers': 'POR',
    'Wizards': 'WAS',
    'Mavericks': 'DAL',
    'Nuggets': 'DEN',
    'Kings': 'SAC',
    'Magic': 'ORL',
    'Cavaliers': 'CLE',
    'Knicks': 'NYK',
    'Bucks': 'MIL',
    'Hornets': 'CHA',
    'Bulls': 'CHI',
    'Grizzlies': 'MEM',
    'Timberwolves': 'MIN',
    'Spurs': 'SAS',
    'Suns': 'PHX',
    'Pacers': 'IND',
    'Nets': 'BKN',
    'Thunder': 'OKC',
    'Warriors': 'GSW'
}
df['homeTeam']=df['homeTeam'].map(team_acronyms)

df['awayTeam']=df['awayTeam'].map(team_acronyms)

df =df[['gameId','gameDate','homeTeam','awayTeam']]

df.rename(columns={'gameId':'game_id','gameDate':'game_date','homeTeam':'home_team','awayTeam':'away_team'},inplace=True)
old_dates=pd.read_csv('../web_app/data/game_dates.csv')


old_map=dict(zip(old_dates.team,old_dates.TEAM_ID))

df['home_team_id']=df['home_team'].map(old_map)

df['away_team_id']=df['away_team'].map(old_map)
df.dropna(subset= 'home_team_id',inplace=True)
df.to_csv('schedule.csv')
df.to_csv('../web_app/data/schedule.csv')
df['home_team_id'].unique()

dict_keys(['meta', 'leagueSchedule'])
          gameId         gameCode  gameStatus gameStatusText  \
1362  0042400156  20250502/HOUGSW           1            TBD   
1363  0042400166  20250502/LALMIN           1            TBD   
1364  0042400117  20250503/ORLBOS           1            TBD   
1365  0042400127  20250503/DETNYK           1            TBD   
1366  0042400147  20250503/MEMOKC           1            TBD   
1367  0042400177  20250503/LACDEN           1            TBD   
1368  0042400107  20250504/MIACLE           1            TBD   
1369  0042400137  20250504/MILIND           1            TBD   
1370  0042400157  20250504/GSWHOU           1            TBD   
1371  0042400167  20250504/MINLAL           1            TBD   

                 gameDate              arenaName      arenaCity      homeTeam  \
1362  05/02/2025 00:00:00           Chase Center  San Francisco      Warriors   
1363  05/02/2025 00:00:00          Target Center    Minneapolis  Timberwolves   
1364  05/03/20

array([1.61061274e+09, 1.61061276e+09, 1.61061275e+09, 1.61061275e+09,
       1.61061274e+09, 1.61061277e+09, 1.61061276e+09, 1.61061276e+09,
       1.61061274e+09, 1.61061276e+09, 1.61061274e+09, 1.61061276e+09,
       1.61061274e+09, 1.61061274e+09, 1.61061275e+09, 1.61061276e+09,
       1.61061276e+09, 1.61061276e+09, 1.61061275e+09, 1.61061276e+09,
       1.61061275e+09, 1.61061274e+09, 1.61061276e+09, 1.61061274e+09,
       1.61061275e+09, 1.61061275e+09, 1.61061275e+09, 1.61061274e+09,
       1.61061276e+09, 1.61061275e+09])

In [4]:
df.tail(40)

,game_id,game_date,home_team,away_team,home_team_id,away_team_id
1332,0042400123,2025-04-24,DET,NYK,1.610613e+09,1.610613e+09
1333,0042400143,2025-04-24,MEM,OKC,1.610613e+09,1.610613e+09
1334,0042400173,2025-04-24,LAC,DEN,1.610613e+09,1.610613e+09
1335,0042400113,2025-04-25,ORL,BOS,1.610613e+09,1.610613e+09
1336,0042400133,2025-04-25,MIL,IND,1.610613e+09,1.610613e+09
1337,0042400163,2025-04-25,MIN,LAL,1.610613e+09,1.610613e+09
1338,0042400103,2025-04-26,MIA,CLE,1.610613e+09,1.610613e+09
1339,0042400144,2025-04-26,MEM,OKC,1.610613e+09,1.610613e+09
1340,0042400153,2025-04-26,GSW,HOU,1.610613e+09,1.610613e+09
1341,0042400174,2025-04-26,LAC,DEN,1.610613e+09,1.610613e+09
